# Introdução

Descrição da atividade.

Imagine agora, que você vai atuar como Expert em Data Analytics em uma empresa que exporta vinhos do Brasil para o mundo todo.

Sua área é recém-criada dentro da empresa, e você será responsável pelos relatórios iniciais a serem apresentados em uma reunião de investidores e acionistas, explicando a quantidade de vinhos exportados e os fatores externos que podem vir a surgir e que interferem nas análises:

- 	Dados climáticos.
- 	Dados demográficos.
- 	Dados econômicos.
- 	Dados de avaliações de vinhos.
  
O Head de Dados pediu para que você construísse uma tabela contendo as seguintes informações:
-   País de origem (Brasil).
- 	País de destino.
- 	Quantidade em litros de vinho exportado (utilize: 1KG =1L).
- 	Valor em US$.

Os dados que lhe forneceram são de uma vinícola parceira, e podem ser encontrados  [aqui](http://vitibrasil.cnpuv.embrapa.br/index.php?opcao=opt_01).
Tomando como referência o [relatório](https://ainfo.cnptia.embrapa.br/digital/bitstream/item/215377/1/COMUNICADO-TECNICO-214-Publica-602-versao-2020-08-14.pdf), sabemos que os valores exportados constantes das bases de dados estão em US$ 1.000,00 (FOB).

Com o intuito de confirmar tal informação, ainda extraímos os dados diretamente da plataforma [Comexstat](http://comexstat.mdic.gov.br/pt/geral/97910) de responsabilidade do Ministério da Indústria, Comércio Exterior e Serviços (MDIC). Nesta caso, os vinhos de mesa correspondem ao produto de Código CUCI 11217 - "Vinhos de uvas frescas (exceto vinho espumante); mostos de uvas cuja fermentação tenha sido impedida ou interrompida por adição de álcool".

Ao compararmos os dados da Comexstat com os dados da Vitibrasil, vemos que o valor exportado está expresso em US$ 1.000,00 (FOB).


https://balanca.economia.gov.br/balanca/manual/Manual.pdf

https://ourworldindata.org

https://data.worldbank.org

 Classificação Uniforme para o Comércio Internacional - CUCI, também conhecida
como Standard International Trade Classification - SITC é uma classificação de produto da ONU
usada para estatísticas de comércio exterior (valores de exportação e importação e volumes de 
mercadorias), permitindo comparações internacionais de mercadorias e produtos
manufaturados.


O valor FOB indica o preço da mercadoria em dólares americanos sob o Incoterm FOB
(Free on Board), modalidade na qual o vendedor é responsável por embarcar a mercadoria
enquanto o comprador assume o pagamento do frete, seguros e demais custos pós embarque.
Nesse caso, o valor informado da mercadoria expressa o valor exclusivamente da mercadoria.


Medida que expressa o peso líquido da mercadoria. Mesmo produtos com quantidades
estatísticas diferentes do quilograma também possuem disponível a medida em quilograma,
referindo-se ao peso líquido da mercadoria, ou seja, mercadoria desconsiderando embalagens,
caixas ou quaisquer outros adicionais de transporte. Vale relembrar que essa informação, bem
como as demais informadas nas operações de comércio exterior, é de livre preenchimento e de
responsabilidade exclusiva dos operadores de comércio exterior.


- opções de ferramentas:
  - https://ipyvizzu-story.vizzuhq.com/latest/
  - http://gsap.com/

# Análise de dados

In [1]:
# importa as bibliotecas e carrega os dados
import plotly.express as px
import polars as pl

# carrega os dados de importação de vinhos
exp_vinho = (
    pl.read_csv("../data/ExpVinho.csv", separator=";")
    .rename({"País": "country"})
    .drop("Id")
)

# carrega os dados com as colunas necessárias para posterior plotagem dos dados em mapa
df = px.data.gapminder()

geo_info = (
    df.groupby(["country", "iso_alpha"])
    .first()
    .reset_index()[["country", "iso_alpha", "continent"]]
)

exp_vinho = pl.DataFrame.join(pl.from_dataframe(geo_info), exp_vinho, on="country")

In [2]:
import polars.selectors as cs

# dados de quantidade exportada
exp_vinho_qty = exp_vinho.select(
    cs.by_name("country")
    | cs.by_name("iso_alpha")
    | cs.by_name("continent")
    | cs.ends_with("_0")
)
exp_vinho_qty.columns = [
    col.replace("_duplicated_0", "") for col in exp_vinho_qty.columns
]

# dados de exportação em valor (US$)
exp_vinho_val = exp_vinho.select(
    cs.by_name("country")
    | cs.by_name("iso_alpha")
    | cs.by_name("continent")
    | ~cs.ends_with("_0")
)

In [5]:
# dataframe com US$ exportado/ litro
exp_vinho_val.head()

country,iso_alpha,continent,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
str,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
"""Angola""","""AGO""","""Africa""",0,0,0,0,0,0,0,0,0,3465756,1682244,0,0,0,0,339,774,1316,18500,168,930,125,292,189,0,24,2118,68494,15780,10998,249717,12150,9812,19937,37573,24056,3766,25931,25721,54786,33557,13889,2833,1573,12182,1908,7359,10170,477,345,0,0,4068
"""Argentina""","""ARG""","""Americas""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,360,0,6300,0,1125,322990,691422,0,2100,40,211,0,0,1477,0,0,0,0,0,0,0,162,0,13253,0,0,20385,0,0,0,15711,0,1015,6,480
"""Bangladesh""","""BGD""","""Asia""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,2,7
"""Benin""","""BEN""","""Africa""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10350,0,10350,0,0,0,0,0,0,0,5040,0,0,9,0,0,0
"""Chile""","""CHL""","""Americas""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,900,0,0,0,160,0,0,0,0,0,0,0,0,31,0,0,0,1988,60930,0,1475,0,0,8550,0,0,0,0,0,11049,0,11,26,2094
